In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import sys
import os
PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.path.dirname('utils'), 
                  os.pardir)
)
sys.path.append(PROJECT_ROOT)
import utils
%matplotlib qt

In [10]:




def create_map_array(decimal_places = 1, additional_border=100):
    '''Creates a map array for the distance map of the street map. 
    decimal_places: how many decimal places of the original coordinates should be considered
    additional_border: how big should the border around the actual map should be
    '''
    to_array_indices = 10**decimal_places
    to_coordinates = 10**-decimal_places
    lines, arcs= utils.load_map('road_points_data')
    road_points = np.concatenate([lines, arcs])
    road_point_indicies = (np.round(road_points,decimal_places)*to_array_indices).astype(int)
    x_min_bound = road_point_indicies[:,0].min() - additional_border
    y_min_bound = road_point_indicies[:,1].min() - additional_border
    x_max_bound = road_point_indicies[:,0].max() + additional_border
    y_max_bound = road_point_indicies[:,1].max() + additional_border
    size_x = int(x_max_bound - x_min_bound)
    size_y = int(y_max_bound - y_min_bound)
    map_array = np.zeros((y_max_bound + (0 - y_min_bound), x_max_bound + (0 -x_min_bound)))
    
    return map_array, (x_min_bound, x_max_bound), (y_min_bound,y_max_bound), (size_x, size_y)

In [28]:
def convert_coord_zero(point):
    '''Converts a coordinate from ingame coodrinates to map coordinates'''
    return point + np.array([0-(x_min_bound + additional_border), 0-(y_min_bound+additional_border)]) + np.array([additional_border/2, additional_border/2])

In [31]:
converted_points = []
for p in road_point_indicies: 
    try: 
        converted_point = convert_coord_zero(p)
        map_array[converted_point[1], converted_point[0]] = 1
        converted_points.append(converted_point)
    except: 
        print("Index out of bounce " + str(converted_point))
        break
converted_points = np.array(converted_points)

Index out of bounce [2222. 1504.]


In [30]:
plt.imshow(map_array, cmap='gray')

In [19]:
norm = lambda vec: np.sqrt(vec[0]**2 + vec[1]**2)

In [14]:
current_point_array = np.array([10,100])
substraction_results = current_point_array-converted_points[0:100]
distances = np.apply_along_axis(np.linalg.norm, 1, substraction_results)

In [23]:
def clamp(n, smallest, largest): 
    return max(smallest, min(n, largest))


In [33]:
empty_map_array = np.zeros_like(map_array)
smallest = 0
largest = np.linalg.norm((np.array([0,0])-np.array(map_array.shape)))
for y in range(0,map_array.shape[0]):
    for x in range(0,map_array.shape[1]):
        current_point = np.array([x,y])
        substraction_results = current_point-converted_points[0:100]
        distance = clamp(np.apply_along_axis(np.linalg.norm, 1, substraction_results).min(), smallest, largest)
        empty_map_array[y,x] = distance


In [35]:
plt.imshow(empty_map_array, cmap='gray')

In [31]:
empty_map_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])